# Αναγνώριση Προτύπων Μηχανικής Μάθησης
Εργασία 2 \\
Χρήστος Μπριστογιάννης sdi1900129 \\

##Ερώτημα 1: Αναγνώριση Προσώπων (Face recognition)

###Ζητούμενα i
Να γράψετε μία συνάρτηση loadImages(path, set_number) η οποία παίρνει ως είσοδο το
path στο οποίο βρίσκεται ο φάκελος των εικόνων π.χ. loadImages(“C:/images”, “Set_1”),
διαβάζει τις εικόνες και επιστέφει έναν πίνακα δεδομένων ανάλογα με το set_number,
όπου κάθε εικόνα αναπαρίσταται ως διάνυσμα στήλη. Η συνάρτηση επιστέφει επίσης τις
κατηγορίες (labels) στις οποίες ανήκουν οι διαφορετικές εικόνες κωδικοποιημένες με
ακεραίους (π.χ. 0 για φωτογραφίες που ανήκουν στο person_0, 1 για τις φωτογραφίες
που ανήκουν στο person_1 κτλ).

In [ ]:
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

def loadImages(path, set_number):
  if set_number == "Set_1":
    start = 1
    end = 7
  elif set_number == "Set_2":
    start = 8
    end = 19
  elif set_number == "Set_3":
    start = 20
    end = 31
  elif set_number == "Set_4":
    start = 32
    end = 45
  elif set_number == "Set_5":
    start = 46
    end = 64
  else:
    print("Error")
    return

  arr = []
  labels = []
  for i in range(1, 11, 1):
    for j in range(start, end+1, 1):
      if i == 10:
        mes = 'person10'+'_'
      else:
        mes = 'person0'+str(i)+'_'
      if j < 10:
        mes = mes+'0'+str(j)
      else:
        mes = mes + str(j)

      img = cv2.imread(path+ mes + '.png', 0)

      arr.append(np.reshape(img,-1))
      labels.append(i)
  return arr, labels


#load images
numberOfSets = 5
nmsset = ["Set_1", "Set_2", "Set_3", "Set_4", "Set_5"]
imgset = [None] * numberOfSets
lblset = [None] * numberOfSets
path = '/content/drive/My Drive/faces/'

for i in range(0, numberOfSets):
  imgset[i], lblset[i] = loadImages(path, nmsset[i])
  # print(imgset[i])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Ζητούμενα ii
Να εκπαιδεύσετε την μέθοδο Eigenfaces με d = 9 και d = 30 χρησιμοποιώντας όλες τις
εικόνες στο Set_1 (70 εικόνες) και να αναγνωρίσετε τα πρόσωπα στα Set_1 έως Set_5.
Για κάθε Set και κάθε τιμή της διάστασης d να αναφέρετε την ακρίβεια ταξινόμησης. Για
το Set_1 αναμένουμε 100% ακρίβεια ταξινόμησης καθώς χρησιμοποιήθηκε για την
εκπαίδευση της μεθόδου Eigenfaces. Σχολιάστε την δυνατότητα γενίκευσης της μεθόδου
στα διαφορετικά Sets.

In [ ]:
from sklearn.decomposition import PCA


n_components = 9 ## how may componets to keep
pca = PCA(n_components=n_components, whiten=True).fit(imgset[4])

print ("\n--pca--\n",pca)


components = pca.transform(imgset[4]) ## tranform the data to the new axes system)
print ("components.shape",components.shape)


--pca--
 PCA(n_components=9, whiten=True)
components.shape (190, 9)


##Ερώτημα 2: Ταξινόμηση εικόνων χρησιμοποιώντας SVMs

In [2]:
from sklearn.datasets import fetch_openml

# Φόρτωσει δεδομένων
mnist = fetch_openml('mnist_784')

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
import numpy as np
# Μετατρέπουμε τις εικόνες σε μορφή διανύσματος διαστάσεων 28 x 28
X = np.array(mnist.data)
X = X.reshape(-1, 28, 28)

# Κανονικοποιούμε τα δεδομένα στο διάστημα [0,1]
X = X / 255.0

# Split the dataset into training, validation, and test sets
train_images, train_labels = X[:5000], mnist.target[:5000]
val_images, val_labels = X[5000:6000], mnist.target[5000:6000]
test_images, test_labels = X[6000:], mnist.target[6000:]

# print(len(train_images))
# print(len(val_images))
# print(len(test_images))

In [4]:
from sklearn import svm
from sklearn.metrics import accuracy_score

# Define the hyperparameters to test
c_values = [0.01, 0.1, 1, 10]
gamma_values = [0.001, 0.01, 0.1, 1]

# Train and evaluate SVMs with different hyperparameter combinations
for kernel in ['linear', 'rbf']:
    best_accuracy = 0
    best_c = None
    best_gamma = None
    
    print(f'Testing {kernel} kernel...')
    
    for c in c_values:
        for gamma in gamma_values:
            # Train SVM
            svm_model = svm.SVC(kernel=kernel, C=c, gamma=gamma)
            svm_model.fit(train_images.reshape(train_images.shape[0], -1), train_labels)
            
            # Evaluate SVM on validation set
            val_predictions = svm_model.predict(val_images.reshape(val_images.shape[0], -1))
            val_accuracy = accuracy_score(val_labels, val_predictions)
            print(f'For values c: {c} and gamma: {gamma} and accuracy is: {val_accuracy}')
            # Update best hyperparameters if accuracy is higher
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_c = c
                best_gamma = gamma
    
    # Train SVM with best hyperparameters on full training set
    svm_model = svm.SVC(kernel=kernel, C=best_c, gamma=best_gamma)
    svm_model.fit(train_images.reshape(train_images.shape[0], -1), train_labels)
    
    # Evaluate SVM on test set
    test_predictions = svm_model.predict(test_images.reshape(test_images.shape[0], -1))
    test_accuracy = accuracy_score(test_labels, test_predictions)
    
    # Print results
    print(f'Best hyperparameters for {kernel} kernel: C={best_c}, gamma={best_gamma}')
    print(f'Validation accuracy: {best_accuracy:.4f}')
    print(f'Test accuracy: {test_accuracy:.4f}\n')


Testing linear kernel...
For values c: 0.01 and gamma: 0.001 and accuracy is: 0.919
For values c: 0.01 and gamma: 0.01 and accuracy is: 0.919
For values c: 0.01 and gamma: 0.1 and accuracy is: 0.919
For values c: 0.01 and gamma: 1 and accuracy is: 0.919
For values c: 0.1 and gamma: 0.001 and accuracy is: 0.93
For values c: 0.1 and gamma: 0.01 and accuracy is: 0.93
For values c: 0.1 and gamma: 0.1 and accuracy is: 0.93
For values c: 0.1 and gamma: 1 and accuracy is: 0.93
For values c: 1 and gamma: 0.001 and accuracy is: 0.908
For values c: 1 and gamma: 0.01 and accuracy is: 0.908
For values c: 1 and gamma: 0.1 and accuracy is: 0.908
For values c: 1 and gamma: 1 and accuracy is: 0.908
For values c: 10 and gamma: 0.001 and accuracy is: 0.908
For values c: 10 and gamma: 0.01 and accuracy is: 0.908
For values c: 10 and gamma: 0.1 and accuracy is: 0.908
For values c: 10 and gamma: 1 and accuracy is: 0.908
Best hyperparameters for linear kernel: C=0.1, gamma=0.001
Validation accuracy: 0.9300
